In [ ]:
# 필요한 라이브러리 임포트
import torch
import cv2
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')


In [ ]:
# 카메라 설정
cap = cv2.VideoCapture(0)  # USB 카메라 사용
if not cap.isOpened():
    print("Cannot open camera")
    exit()


In [ ]:
import RPi.GPIO as GPIO
import time

GPIO.setmode(GPIO.BCM)
GPIO.setup(17, GPIO.OUT)  # 예시: 오른쪽 모터
GPIO.setup(18, GPIO.OUT)  # 예시: 왼쪽 모터

def move_right():
    GPIO.output(17, GPIO.HIGH)
    GPIO.output(18, GPIO.LOW)
    time.sleep(1)
    GPIO.output(17, GPIO.LOW)

def move_left():
    GPIO.output(17, GPIO.LOW)
    GPIO.output(18, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(18, GPIO.LOW)

def stop():
    GPIO.output(17, GPIO.LOW)
    GPIO.output(18, GPIO.LOW)


In [ ]:
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        
        results = model(frame)


        detected_objects = results.pandas().xyxy[0]


        for index, row in detected_objects.iterrows():
            if row['name'] in ['person', 'car', 'bicycle']:  # 장애물로 간주할 객체는 해보고 정하기 벽, 코너 등등 학습시켜서 해야 할지도 모름
                x_center = (row['xmin'] + row['xmax']) / 2
                if x_center < frame.shape[1] / 3:
                    print("Obstacle detected on the left, moving right")
                    move_right()
                elif x_center > 2 * frame.shape[1] / 3:
                    print("Obstacle detected on the right, moving left")
                    move_left()
                else:
                    print("Obstacle detected in the center, stopping")
                    stop()

        
        clear_output(wait=True)
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        display(plt.gcf())

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    cap.release()
    cv2.destroyAllWindows()
    GPIO.cleanup()
